# Мнемотехническая оптимизация

**Цель**: подобрать оптимальное соответствие пар вида «согласная буква — цифр» для .

Представим следующую задачу: мы хотим разработать систему мнемотехнических правил для запоминания длинных чисел через их разбиение на группы по 2-3 цифры и сопоставление таким буквам слов, содержащих 2-3 согласных, которыми мы кодируем абстрактные цифры. Мы хотим оптимизировать выбор таких кодирующих букв с учетом частотности букв и существительных и равномерном распределении цифр.

Разделим буквы на глассные и согласные. К гласным условно добавим й, на этой есть несколько причин:
1) простота восприятия — половина гласных является йотированными
2) возможность использования не только существительных, но и словосочетаний вида прилагательное+существительное, что упрощает запоминание длинных цепочек.

In [1]:
vows = "аиеёоуыэюяй"
cons = "бвгджзклмнпрстфхцчшщ"
len(cons)

20

Каждой цифре мы можем сопоставить даже не одну, а две буквы. В отличие от стандартных приемов, в нашей задаче мы не собираемся сопоставлять звучание или форму букв определенным символам (например, 4 - ч). Мы хотим только сократить избыточное упоминание согласных.

## Частотный словарь

Для нашей задачи нам потребуется список словарных слов, в котором есть разделение слов по частям речи и желательно — оценка ранга частоты их встречаемости в русскоязычных текстах.

In [2]:
import pandas as pd
df = pd.read_csv('freq.csv', sep='\t')
df

,Lemma,PoS,Freq(ipm),R,D,Doc
0,а,conj,8198.0,100,97,32332
1,а,intj,19.8,99,90,757
2,а,part,6.1,59,79,128
3,а,s,2.7,59,85,160
4,аа,intj,1.5,47,80,68
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212


Выделим существительные (s) и прилагательные (a):

In [3]:
dfs = pd.concat([df.loc[df['PoS'] == 's'], df.loc[df['PoS'] == 'a']])

# сразу удалим все составные слова, содержащие дефис
dfs = dfs[dfs["Lemma"].str.contains('-')==False]
dfs

,Lemma,PoS,Freq(ipm),R,D,Doc
3,а,s,2.7,59,85,160
9,абажур,s,4.9,79,90,269
10,аббат,s,1.6,44,79,63
11,аббатство,s,0.9,33,77,40
12,аббревиатура,s,3.5,83,91,239
...,...,...,...,...,...,...
52113,ясноглазый,a,0.8,35,69,50
52115,ясный,a,90.8,100,96,3087
52118,ястребиный,a,0.4,21,76,33
52124,ячеистый,a,0.7,24,72,34


Для описаннной задачи разбиения в 2-3 символа нам необходимы двух и трехбуквенные соответствия, то есть всего лишь порядка 1000 слов. Однако среди словарных нас в первую очередь интересуют те, что содержат 2 или 3 согласных, которые могут быть окружены любым количеством гласных. Опишем фильтр с помощью соответствующего регулярного выражения.

In [4]:
import re
pattern = re.compile(fr"([{vows}]*[{cons}]){{2,3}}[{vows}]*\b")
dfs23 = dfs[dfs.Lemma.str.match(pattern)]
dfs23

,Lemma,PoS,Freq(ipm),R,D,Doc
9,абажур,s,4.9,79,90,269
10,аббат,s,1.6,44,79,63
13,абвер,s,2.7,22,38,25
16,абзац,s,10.2,99,87,478
24,аборт,s,8.3,93,87,300
...,...,...,...,...,...,...
52077,яркий,a,93.9,100,97,3664
52093,яровой,a,1.4,44,79,66
52102,ярчайший,a,2.0,77,91,153
52115,ясный,a,90.8,100,96,3087


Теперь мы можем сопоставить каждому слову числовой код и сравнить, при каком коде происходит наиболее полное покрытие. Для упрощения кодирования добавим столбец, в котором в словах оставлены одни согласные. Часть строк при этом начнет совпадать, так как в таком дополнительном столбце теряется информация о гласных.

In [5]:
dfs23['cons'] = dfs23.Lemma.str.replace(fr'[{vows}]', '', regex=True)
dfs23.drop_duplicates(subset=['cons'], inplace=True)
dfs23

/var/folders/15/wwc0zrhx639b82j8t2htcpyw0000gn/T/ipykernel_42858/1263814731.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs23['cons'] = dfs23.Lemma.str.replace(fr'[{vows}]', '', regex=True)
/var/folders/15/wwc0zrhx639b82j8t2htcpyw0000gn/T/ipykernel_42858/1263814731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs23.drop_duplicates(subset=['cons'], inplace=True)


,Lemma,PoS,Freq(ipm),R,D,Doc,cons
9,абажур,s,4.9,79,90,269,бжр
10,аббат,s,1.6,44,79,63,ббт
13,абвер,s,2.7,22,38,25,бвр
16,абзац,s,10.2,99,87,478,бзц
24,аборт,s,8.3,93,87,300,брт
...,...,...,...,...,...,...,...
51233,щуплый,a,2.5,69,89,164,щпл
51236,щучий,a,0.9,49,87,65,щч
51280,эдипов,a,0.4,23,79,26,дпв
52009,языковой,a,10.6,91,84,466,зкв


Проверим, что в стобце есть только 2 или 3 согласных.

In [6]:
dfs23['cons'].apply(len).describe()

count    2457.000000
mean        2.873016
std         0.333023
min         2.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         3.000000
Name: cons, dtype: float64

Проверим, что сочетания согласных уникальны:

In [7]:
dfs23['cons'].describe()

count     2457
unique    2457
top        бжр
freq         1
Name: cons, dtype: object

In [8]:
dfs23

,Lemma,PoS,Freq(ipm),R,D,Doc,cons
9,абажур,s,4.9,79,90,269,бжр
10,аббат,s,1.6,44,79,63,ббт
13,абвер,s,2.7,22,38,25,бвр
16,абзац,s,10.2,99,87,478,бзц
24,аборт,s,8.3,93,87,300,брт
...,...,...,...,...,...,...,...
51233,щуплый,a,2.5,69,89,164,щпл
51236,щучий,a,0.9,49,87,65,щч
51280,эдипов,a,0.4,23,79,26,дпв
52009,языковой,a,10.6,91,84,466,зкв


## Поиск оптимума

Далее мы должны создать функцию, которая будет сопоставлять букве цифру. Вполне вероятно, что лучшие комбинации достигаются при распределении отличном от парного разбиения. При рассмотрении соответствия всех согласных получится $10^{20}$, что явно бессмысленно для нашей задачи. Если не утяжелять задачу, можно просто использовать случайные последовательности и максимизировать параметр уникальности получаемых чисел.

In [11]:
import random

digs = 2*list(range(0, 10))

def generate_table():
    random.shuffle(digs)
    table = {c: str(d) for (c, d) in zip (cons, digs)}
    return table

table = generate_table()
table

{'щ': '6',
 'ш': '8',
 'ч': '8',
 'ц': '1',
 'х': '5',
 'ф': '0',
 'т': '2',
 'с': '9',
 'р': '7',
 'п': '7',
 'н': '1',
 'м': '3',
 'л': '4',
 'к': '2',
 'з': '6',
 'ж': '9',
 'д': '3',
 'г': '4',
 'в': '0',
 'б': '5'}

Напишем две функции:
1) функцию преобразования слова к запоминаемой последовательности цифр,
2) функцию, результат которой мы хотим максимизировать

In [12]:
def translate(word, table):
    digit_symbols = []
    for letter in word:
        digit_symbols.append(table.get(letter))
    return "".join(digit_symbols)

def uniqueness(df, table):
    return df['cons'].apply(lambda x: translate(x, table)).nunique()


uniqueness(dfs23, table)

907

In [14]:
old_u = 2
best_table = table

while True:
    try:
        table = generate_table()
        new_u = uniqueness(dfs23, table)
        if new_u > old_u:
            old_u = new_u
            best_table = table
            print(new_u, "".join(table.values()))
    except KeyboardInterrupt:
        break

# 1012 75318298205740646931

835 47049260528516179338
916 51476213995038826740
937 51782660930471984352
941 57213473051862094968
945 80154226407133979586
967 87024681409361937552
980 41605893591087232764
994 41827370824561063995
995 04167946159730258328
1011 28753493468206107915
1012 75318298205740646931


In [ ]:
best_table

# Отображение текущего перевода


In [15]:
dfs23['dig_string'] = dfs23['cons'].apply(lambda x: translate(x, best_table))
dfs23.sort_values(by='dig_string', inplace=True)
dfs23


/var/folders/15/wwc0zrhx639b82j8t2htcpyw0000gn/T/ipykernel_42858/4131098003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs23['dig_string'] = dfs23['cons'].apply(lambda x: translate(x, best_table))
/var/folders/15/wwc0zrhx639b82j8t2htcpyw0000gn/T/ipykernel_42858/4131098003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs23.sort_values(by='dig_string', inplace=True)


,Lemma,PoS,Freq(ipm),R,D,Doc,cons,dig_string
29101,папа,s,143.4,100,91,2079,пп,00
15600,кап,s,1.7,46,73,63,кп,00
15410,какао,s,2.0,67,89,121,кк,00
27117,опека,s,8.0,94,81,428,пк,00
30517,пикап,s,1.6,46,82,70,пкп,000
...,...,...,...,...,...,...,...,...
52119,ятаган,s,0.6,24,75,37,тгн,995
45065,татами,s,1.0,27,64,33,ттм,997
46679,тягота,s,4.9,93,93,336,тгт,999
8013,гогот,s,1.1,49,86,82,ггт,999


In [18]:
dfs23['dig_string'].nunique()

1012

# Подготовка результатов для экспорта в Anki

